In [420]:
import pandas as pd
import sys
sys.path.append('..')
from src.timeconverter import date_to_timestamp


target = pd.read_csv('../train/target/y_predict.csv')
print(len(target['wagnum']), len(target['wagnum'].unique()))

33707 33707


In [421]:
wag_param = pd.read_csv('../preprocessed/wag_param.csv', index_col = 0)
kti = pd.read_csv('../preprocessed/kti.csv', index_col = 0)

train = pd.read_csv('./target/y_train.csv')
train['month'] = train['month'].apply(date_to_timestamp)
test = pd.read_csv('../train2/target/y_test.csv')
test['month'] = test['month'].apply(date_to_timestamp)

In [422]:
features = pd.merge(wag_param, kti, on='wagnum', how='left')

In [423]:
kti

,wagnum,operation_date_dttm,mileage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,axl3_l_w_flange,axl3_r_w_flange,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,325,1659312000,112091,30.5,31.0,26.7,27.4,28.8,28.6,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.800000
1,325,1659484800,112471,30.4,31.4,27.5,27.6,28.9,29.0,25.5,27.8,65.7,65.8,65.7,63.7,61.6,61.4,67.4,69.300000
2,325,1659657600,113938,30.0,31.1,27.1,27.9,28.5,28.9,26.2,27.6,70.8,69.5,65.1,65.6,62.2,60.7,65.7,69.600000
3,325,1663200000,121071,30.5,31.4,26.4,27.3,28.9,28.4,24.9,28.0,70.2,68.1,67.4,65.8,62.3,63.0,70.9,70.300000
4,325,1662681600,117341,30.7,31.5,26.9,27.1,28.9,28.5,25.5,27.5,68.1,64.1,64.5,64.9,61.6,61.1,68.7,62.246842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212300,26543,1677542400,18361,29.5,28.2,27.8,29.0,28.2,29.3,29.7,28.9,64.4,65.7,58.8,58.3,50.1,50.1,58.1,57.100000
212327,33975,1677542400,13736,25.9,28.5,28.4,28.9,28.1,28.0,28.2,29.5,51.4,51.6,46.5,47.3,52.4,50.9,54.0,56.700000
212347,28227,1677542400,154365,26.9,28.5,26.7,28.1,29.0,27.4,29.8,29.0,72.4,72.3,60.2,58.7,69.8,72.1,76.2,78.000000
212349,33698,1677542400,39226,29.4,28.3,27.9,27.0,28.0,28.8,26.7,28.9,35.1,36.6,40.1,40.5,46.6,47.0,43.5,41.700000


In [424]:
train

,wagnum,month,target_month,target_day
0,33361,1672531200,0,0
1,33364,1672531200,0,0
2,33366,1672531200,0,0
3,33358,1672531200,0,0
4,33349,1672531200,0,0
...,...,...,...,...
203848,25045,1669852800,0,0
203849,27156,1669852800,0,0
203850,21361,1669852800,0,0
203851,8061,1669852800,0,0


In [425]:
df = pd.merge(train, kti, how='left', on='wagnum')
df

,wagnum,month,target_month,target_day,operation_date_dttm,mileage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,33361,1672531200,0,0,1.665965e+09,158728.0,31.2,31.2,26.1,29.1,...,30.3,30.0,70.1,67.4,69.8,67.6,61.2,59.9,59.1,58.8
1,33361,1672531200,0,0,1.670112e+09,84.0,29.7,29.0,32.2,31.9,...,28.5,29.8,61.3,60.6,65.7,64.1,57.3,57.1,57.1,54.5
2,33361,1672531200,0,0,1.671754e+09,3108.0,29.6,29.7,32.7,31.9,...,28.1,30.8,61.6,61.2,66.0,65.1,60.2,58.5,57.4,57.0
3,33361,1672531200,0,0,1.671840e+09,4152.0,29.0,29.7,32.1,32.4,...,28.2,31.1,62.0,61.3,65.3,64.8,57.2,58.8,56.9,56.9
4,33361,1672531200,0,0,1.671926e+09,4938.0,29.4,29.1,32.4,32.1,...,28.2,30.4,60.4,60.0,65.1,62.7,58.2,56.1,56.4,53.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699214,21361,1669852800,0,0,1.676074e+09,32714.0,30.8,29.0,28.4,30.3,...,30.0,29.0,65.9,67.1,63.7,63.0,56.2,55.6,58.5,57.3
699215,21361,1669852800,0,0,1.676678e+09,34008.0,29.5,29.0,28.0,29.6,...,29.5,28.7,65.9,68.9,63.2,65.0,55.8,57.2,57.1,58.1
699216,21361,1669852800,0,0,1.676938e+09,35560.0,29.4,28.4,27.8,30.0,...,28.8,29.2,66.5,68.3,63.5,63.6,58.0,56.4,58.0,58.9
699217,8061,1669852800,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [426]:
# Find rows where 'month' is greater than 'operation_date_dttm'
filtered_df = df[df['month'] >= df['operation_date_dttm']]

# Filter to keep rows where 'operation_date_dttm' is the maximum within 'wagnum' groups
filtered_df = filtered_df[filtered_df.groupby(['wagnum', 'month'])['operation_date_dttm'].transform('max') == filtered_df['operation_date_dttm']]
filtered_df


,wagnum,month,target_month,target_day,operation_date_dttm,mileage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
5,33361,1672531200,0,0,1.672099e+09,6102.0,29.1,29.3,31.8,32.3,...,27.6,30.2,61.1,61.7,64.6,65.4,58.1,58.3,56.6,56.1
40,33364,1672531200,0,0,1.670198e+09,156191.0,26.7,28.1,30.2,29.1,...,28.4,29.9,72.0,70.7,72.9,74.4,71.0,72.4,72.8,71.5
60,33366,1672531200,0,0,1.668211e+09,156730.0,27.9,27.9,27.4,29.6,...,28.5,29.3,73.0,71.1,74.7,73.8,73.9,73.2,69.6,69.0
72,33358,1672531200,0,0,1.671667e+09,130991.0,27.8,27.8,28.3,28.6,...,31.0,29.3,53.6,52.9,45.3,45.1,51.6,53.3,58.2,56.0
88,33373,1672531200,0,0,1.671840e+09,4409.0,32.0,29.7,29.5,31.5,...,29.4,30.0,50.1,48.6,47.8,47.5,60.2,59.6,60.6,60.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699090,22774,1669852800,0,0,1.668643e+09,77548.0,30.3,28.5,29.9,30.8,...,30.6,31.3,69.0,69.2,62.2,65.7,66.3,66.8,66.5,70.2
699121,13706,1669852800,0,0,1.668470e+09,105520.0,29.6,29.9,26.9,30.1,...,29.1,30.0,74.3,72.7,76.2,73.7,75.4,71.2,72.8,70.6
699167,10505,1669852800,0,0,1.669853e+09,96755.0,28.7,28.7,26.8,27.5,...,27.5,29.0,68.4,67.9,63.8,65.0,56.8,60.2,61.6,62.2
699182,31340,1669852800,0,0,1.669162e+09,78905.0,29.2,29.2,25.1,28.8,...,27.2,29.1,67.4,68.5,65.0,65.8,62.9,65.7,65.8,67.0


In [366]:
features.info()

cat_features = ['model','zavod_build', 'date_iskl', 'kuzov', 'telega', 'tormoz', 'tipvozd']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95115 entries, 0 to 95114
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   wagnum               95115 non-null  int64  
 1   model                95115 non-null  object 
 2   rod_id               95115 non-null  int64  
 3   gruz                 95115 non-null  int64  
 4   cnsi_gruz_capacity   95115 non-null  int64  
 5   cnsi_volumek         95115 non-null  float64
 6   tara                 95115 non-null  int64  
 7   date_build           95115 non-null  int64  
 8   srok_sl              95115 non-null  int64  
 9   zavod_build          95115 non-null  int64  
 10  date_iskl            95115 non-null  int64  
 11  cnsi_probeg_dr       95115 non-null  int64  
 12  cnsi_probeg_kr       95115 non-null  int64  
 13  kuzov                95115 non-null  int64  
 14  telega               95115 non-null  int64  
 15  tormoz               95115 non-null 

In [367]:
nan_cols = ['axl1_l_w_flange',
       'axl1_r_w_flange', 'axl2_l_w_flange', 'axl2_r_w_flange',
       'axl3_l_w_flange', 'axl3_r_w_flange', 'axl4_l_w_flange',
       'axl4_r_w_flange', 'axl1_l_w_rim', 'axl1_r_w_rim', 'axl2_l_w_rim',
       'axl2_r_w_rim', 'axl3_l_w_rim', 'axl3_r_w_rim', 'axl4_l_w_rim',
       'axl4_r_w_rim']


def preprocessing(data, test=False):
    
    data = pd.merge(data, features, on='wagnum', how='left')
    data['operation_date_dttm'].fillna(data['month'], inplace=True)
    data[data['month'] >= data['operation_date_dttm']]
    means_nan_cols = data[nan_cols].mean()
    mileage_mean = round(data['mileage_all'].mean())
    data['mileage_all'].fillna(mileage_mean, inplace=True)
    data[nan_cols] = data[nan_cols].fillna(means_nan_cols)

    
    return data

In [368]:
train = preprocessing(train)

print(train.isna().sum().sum())
train

0


,wagnum,month,target_month,target_day,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,33361,1672531200,0,0,12-1303-01,1,690,690,88.0,245,...,30.300000,30.000000,70.100000,67.400000,69.800000,67.600000,61.200000,59.90000,59.100000,58.8000
1,33361,1672531200,0,0,12-1303-01,1,690,690,88.0,245,...,28.500000,29.800000,61.300000,60.600000,65.700000,64.100000,57.300000,57.10000,57.100000,54.5000
2,33361,1672531200,0,0,12-1303-01,1,690,690,88.0,245,...,28.100000,30.800000,61.600000,61.200000,66.000000,65.100000,60.200000,58.50000,57.400000,57.0000
3,33361,1672531200,0,0,12-1303-01,1,690,690,88.0,245,...,28.200000,31.100000,62.000000,61.300000,65.300000,64.800000,57.200000,58.80000,56.900000,56.9000
4,33361,1672531200,0,0,12-1303-01,1,690,690,88.0,245,...,28.200000,30.400000,60.400000,60.000000,65.100000,62.700000,58.200000,56.10000,56.400000,53.8000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570676,21361,1669852800,0,0,12-783,1,700,700,76.0,234,...,29.400000,28.100000,68.500000,66.500000,64.400000,65.400000,57.900000,56.40000,59.900000,59.7000
570677,21361,1669852800,0,0,12-783,1,700,700,76.0,234,...,29.200000,28.300000,67.300000,68.300000,64.600000,64.400000,57.200000,56.70000,59.000000,59.3000
570678,21361,1669852800,0,0,12-783,1,700,700,76.0,234,...,29.400000,29.200000,67.800000,68.400000,65.300000,63.900000,57.700000,55.20000,59.100000,56.2000
570679,8061,1669852800,0,0,11-286,0,670,670,138.0,270,...,29.166353,29.218642,62.373331,62.386428,61.318671,61.344268,61.346119,61.35398,62.379559,62.3867


In [369]:
test = preprocessing(test)
print(test.isna().sum().sum())
test

0


,wagnum,month,target_month,target_day,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,33361,1675209600,0,0,12-1303-01,1,690,690,88.0,245,...,30.300000,30.000000,70.100000,67.400000,69.800000,67.600000,61.20000,59.900000,59.100000,58.800000
1,33361,1675209600,0,0,12-1303-01,1,690,690,88.0,245,...,28.500000,29.800000,61.300000,60.600000,65.700000,64.100000,57.30000,57.100000,57.100000,54.500000
2,33361,1675209600,0,0,12-1303-01,1,690,690,88.0,245,...,28.100000,30.800000,61.600000,61.200000,66.000000,65.100000,60.20000,58.500000,57.400000,57.000000
3,33361,1675209600,0,0,12-1303-01,1,690,690,88.0,245,...,28.200000,31.100000,62.000000,61.300000,65.300000,64.800000,57.20000,58.800000,56.900000,56.900000
4,33361,1675209600,0,0,12-1303-01,1,690,690,88.0,245,...,28.200000,30.400000,60.400000,60.000000,65.100000,62.700000,58.20000,56.100000,56.400000,53.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94591,21361,1675209600,0,0,12-783,1,700,700,76.0,234,...,29.300000,28.700000,68.800000,69.000000,66.200000,64.300000,58.80000,56.700000,59.900000,59.400000
94592,21361,1675209600,0,0,12-783,1,700,700,76.0,234,...,29.400000,28.100000,68.500000,66.500000,64.400000,65.400000,57.90000,56.400000,59.900000,59.700000
94593,21361,1675209600,0,0,12-783,1,700,700,76.0,234,...,29.200000,28.300000,67.300000,68.300000,64.600000,64.400000,57.20000,56.700000,59.000000,59.300000
94594,21361,1675209600,0,0,12-783,1,700,700,76.0,234,...,29.400000,29.200000,67.800000,68.400000,65.300000,63.900000,57.70000,55.200000,59.100000,56.200000


In [370]:
def split(data):
    y1 = data['target_month']
    y2 = data['target_day']

    X = data.drop(['target_month', 'target_day'], axis=1)
    return X, y1, y2

In [371]:
X_train, y1_train, y2_train = split(train)
X_train

,wagnum,month,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,date_build,srok_sl,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,30.300000,30.000000,70.100000,67.400000,69.800000,67.600000,61.200000,59.90000,59.100000,58.8000
1,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.500000,29.800000,61.300000,60.600000,65.700000,64.100000,57.300000,57.10000,57.100000,54.5000
2,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.100000,30.800000,61.600000,61.200000,66.000000,65.100000,60.200000,58.50000,57.400000,57.0000
3,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.200000,31.100000,62.000000,61.300000,65.300000,64.800000,57.200000,58.80000,56.900000,56.9000
4,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.200000,30.400000,60.400000,60.000000,65.100000,62.700000,58.200000,56.10000,56.400000,53.8000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570676,21361,1669852800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.400000,28.100000,68.500000,66.500000,64.400000,65.400000,57.900000,56.40000,59.900000,59.7000
570677,21361,1669852800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.200000,28.300000,67.300000,68.300000,64.600000,64.400000,57.200000,56.70000,59.000000,59.3000
570678,21361,1669852800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.400000,29.200000,67.800000,68.400000,65.300000,63.900000,57.700000,55.20000,59.100000,56.2000
570679,8061,1669852800,11-286,0,670,670,138.0,270,757296000,1760054400,...,29.166353,29.218642,62.373331,62.386428,61.318671,61.344268,61.346119,61.35398,62.379559,62.3867


In [372]:
X_test, y1_test, y2_test = split(train)
X_test

,wagnum,month,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,date_build,srok_sl,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,30.300000,30.000000,70.100000,67.400000,69.800000,67.600000,61.200000,59.90000,59.100000,58.8000
1,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.500000,29.800000,61.300000,60.600000,65.700000,64.100000,57.300000,57.10000,57.100000,54.5000
2,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.100000,30.800000,61.600000,61.200000,66.000000,65.100000,60.200000,58.50000,57.400000,57.0000
3,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.200000,31.100000,62.000000,61.300000,65.300000,64.800000,57.200000,58.80000,56.900000,56.9000
4,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.200000,30.400000,60.400000,60.000000,65.100000,62.700000,58.200000,56.10000,56.400000,53.8000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570676,21361,1669852800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.400000,28.100000,68.500000,66.500000,64.400000,65.400000,57.900000,56.40000,59.900000,59.7000
570677,21361,1669852800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.200000,28.300000,67.300000,68.300000,64.600000,64.400000,57.200000,56.70000,59.000000,59.3000
570678,21361,1669852800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.400000,29.200000,67.800000,68.400000,65.300000,63.900000,57.700000,55.20000,59.100000,56.2000
570679,8061,1669852800,11-286,0,670,670,138.0,270,757296000,1760054400,...,29.166353,29.218642,62.373331,62.386428,61.318671,61.344268,61.346119,61.35398,62.379559,62.3867


In [373]:
# from catboost import CatBoostClassifier
# from sklearn.model_selection import StratifiedKFold

# # Assuming 'X' is your feature matrix and 'y' is your target variable

# # Define the number of folds for cross-validation
# num_folds = 5  # Change this to the desired number of folds

# # Initialize Stratified K-Fold cross-validation
# kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# # Initialize CatBoost Classifier
# model = CatBoostClassifier(iterations=500, depth=16, loss_function='Logloss', random_seed=42, cat_features=cat_features, eval_metric='F1')

# # Perform cross-validation
# for train_index, val_index in kf.split(X, y):
#     X_train, X_val = X.iloc[train_index], X.iloc[val_index]
#     y_train, y_val = y.iloc[train_index], y.iloc[val_index]

#     # Train the model and validate on the current fold
#     model.fit(X_train, y_train, eval_set=(X_val, y_val))


In [374]:
from catboost import CatBoostClassifier

# Initializing CatBoost Classifier
model1 = CatBoostClassifier(iterations=500, loss_function='Logloss', random_seed=42, cat_features=cat_features, eval_metric='F1')
model2 = CatBoostClassifier(iterations=500, loss_function='Logloss', random_seed=42, cat_features=cat_features, eval_metric='F1')

# Training the model
model1.fit(X_train, y1_train)
model2.fit(X_train, y2_train)
# # Model evaluation (if needed)
# metrics = model.get_evals_result()

# Feature importance
feature_importance1 = model1.get_feature_importance()
feature_importance2 = model1.get_feature_importance()

print(feature_importance1)
print(feature_importance2)

Learning rate set to 0.292387
0:	learn: 0.0000000	total: 253ms	remaining: 2m 6s
1:	learn: 0.0000000	total: 477ms	remaining: 1m 58s
2:	learn: 0.0000000	total: 746ms	remaining: 2m 3s
3:	learn: 0.0103630	total: 1.01s	remaining: 2m 5s
4:	learn: 0.0064788	total: 1.22s	remaining: 2m 1s
5:	learn: 0.0099586	total: 1.4s	remaining: 1m 55s
6:	learn: 0.0113564	total: 1.63s	remaining: 1m 54s
7:	learn: 0.0200642	total: 1.82s	remaining: 1m 51s
8:	learn: 0.0200682	total: 2.03s	remaining: 1m 50s
9:	learn: 0.0229532	total: 2.24s	remaining: 1m 49s
10:	learn: 0.0376310	total: 2.46s	remaining: 1m 49s
11:	learn: 0.0384081	total: 2.7s	remaining: 1m 49s
12:	learn: 0.0387868	total: 2.91s	remaining: 1m 49s
13:	learn: 0.0397648	total: 3.1s	remaining: 1m 47s
14:	learn: 0.0315870	total: 3.27s	remaining: 1m 45s
15:	learn: 0.0315817	total: 3.51s	remaining: 1m 46s
16:	learn: 0.0407194	total: 3.75s	remaining: 1m 46s
17:	learn: 0.0433377	total: 3.94s	remaining: 1m 45s
18:	learn: 0.0435299	total: 4.16s	remaining: 1m 45s

In [375]:
y1_predicted = model1.predict(X_test)
y2_predicted = model2.predict(X_test)

In [376]:
from sklearn.metrics import f1_score
score_month = f1_score(y1_predicted, y1_test)
score_day = f1_score(y2_predicted, y2_test)
score = 0.5 * score_month + 0.5 * score_day
score

0.554705339232164

Submission

In [391]:
target['month'] = target['month'].apply(date_to_timestamp)
target = preprocessing(target)
target

,wagnum,month,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,date_build,srok_sl,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,33361,1677628800,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,30.300000,30.000000,70.100000,67.400000,69.800000,67.600000,61.20000,59.900000,59.100000,58.800000
1,33361,1677628800,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.500000,29.800000,61.300000,60.600000,65.700000,64.100000,57.30000,57.100000,57.100000,54.500000
2,33361,1677628800,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.100000,30.800000,61.600000,61.200000,66.000000,65.100000,60.20000,58.500000,57.400000,57.000000
3,33361,1677628800,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.200000,31.100000,62.000000,61.300000,65.300000,64.800000,57.20000,58.800000,56.900000,56.900000
4,33361,1677628800,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.200000,30.400000,60.400000,60.000000,65.100000,62.700000,58.20000,56.100000,56.400000,53.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94590,21361,1677628800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.300000,28.700000,68.800000,69.000000,66.200000,64.300000,58.80000,56.700000,59.900000,59.400000
94591,21361,1677628800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.400000,28.100000,68.500000,66.500000,64.400000,65.400000,57.90000,56.400000,59.900000,59.700000
94592,21361,1677628800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.200000,28.300000,67.300000,68.300000,64.600000,64.400000,57.20000,56.700000,59.000000,59.300000
94593,21361,1677628800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.400000,29.200000,67.800000,68.400000,65.300000,63.900000,57.70000,55.200000,59.100000,56.200000


In [392]:
y1_target = model1.predict(target)
y2_target = model2.predict(target)

In [393]:
target['target_month'] = y1_target
target['target_day'] = y2_target

In [394]:
target = target[['wagnum', 'month', 'target_month', 'target_day']]
target

,wagnum,month,target_month,target_day
0,33361,1677628800,0,0
1,33361,1677628800,0,0
2,33361,1677628800,0,0
3,33361,1677628800,0,0
4,33361,1677628800,0,0
...,...,...,...,...
94590,21361,1677628800,0,0
94591,21361,1677628800,0,0
94592,21361,1677628800,0,0
94593,21361,1677628800,0,0


In [396]:
import pandas as pd

# Your DataFrame 'df' assumed

# Define a custom function to find the most frequent values for 'target_month' and 'target_day'
def most_frequent(group):
    frequent_month = group['target_month'].mode().iloc[0]
    frequent_day = group['target_day'].mode().iloc[0]
    return pd.DataFrame({
        'wagnum': group['wagnum'].iloc[0],
        'month': group['month'].iloc[0],
        'target_month': [frequent_month],
        'target_day': [frequent_day]
    })

# Group by 'month' and 'wagnum', then apply the custom function
reduced_groups = target.groupby(['month', 'wagnum']).apply(most_frequent).reset_index(drop=True)


In [399]:
from src.timeconverter import timestamp_to_date

In [400]:
reduced_groups['month'] = reduced_groups['month'].apply(timestamp_to_date)
reduced_groups

,wagnum,month,target_month,target_day
0,0,2023-03-01,0,0
1,1,2023-03-01,0,0
2,2,2023-03-01,0,0
3,3,2023-03-01,0,0
4,4,2023-03-01,0,0
...,...,...,...,...
33702,33972,2023-03-01,0,0
33703,33973,2023-03-01,0,0
33704,33974,2023-03-01,0,0
33705,33975,2023-03-01,0,0


In [353]:
submission = pd.read_csv('../train/target/y_predict.csv')
submission

,wagnum,month
0,33361,2023-03-01
1,33364,2023-03-01
2,33366,2023-03-01
3,33358,2023-03-01
4,33349,2023-03-01
...,...,...
33702,17621,2023-03-01
33703,25045,2023-03-01
33704,27156,2023-03-01
33705,21361,2023-03-01


In [401]:
ans = pd.merge(submission, reduced_groups, how='left')
ans

,wagnum,month,target_month,target_day
0,33361,2023-03-01,0,0
1,33364,2023-03-01,0,0
2,33366,2023-03-01,0,0
3,33358,2023-03-01,0,0
4,33349,2023-03-01,0,0
...,...,...,...,...
33702,17621,2023-03-01,0,0
33703,25045,2023-03-01,0,0
33704,27156,2023-03-01,0,0
33705,21361,2023-03-01,0,0


In [403]:
ans.to_csv('submission.csv',index=False)